# MNIST - Neural Network
## Xavier initialization
## More deep and wide
## Dropout

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# nn layers
L1 = torch.nn.Linear(784, 512, bias=True)
L2 = torch.nn.Linear(512, 512, bias=True)
L3 = torch.nn.Linear(512, 512, bias=True)
L4 = torch.nn.Linear(512, 512, bias=True)
L5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [7]:
# Xavier initialization
torch.nn.init.xavier_uniform_(L1.weight)
torch.nn.init.xavier_uniform_(L2.weight)
torch.nn.init.xavier_uniform_(L3.weight)
torch.nn.init.xavier_uniform_(L4.weight)
torch.nn.init.xavier_uniform_(L5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [8]:
# model
model = torch.nn.Sequential(L1, relu, dropout,
                            L2, relu, dropout,
                            L3, relu, dropout,
                            L4, relu, dropout,
                            L5).to(device) 

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [10]:
total_batch = len(data_loader)
model.train()  # set train mode (dropout=True)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch: {:2d}, Cost = {:.6f}'.format(epoch + 1, avg_cost))

print('Learning finished')

Epoch:  1, Cost = 0.307182
Epoch:  2, Cost = 0.139162
Epoch:  3, Cost = 0.112666
Epoch:  4, Cost = 0.096059
Epoch:  5, Cost = 0.083635
Epoch:  6, Cost = 0.074380
Epoch:  7, Cost = 0.068586
Epoch:  8, Cost = 0.066356
Epoch:  9, Cost = 0.057277
Epoch: 10, Cost = 0.056394
Epoch: 11, Cost = 0.050565
Epoch: 12, Cost = 0.049677
Epoch: 13, Cost = 0.049100
Epoch: 14, Cost = 0.046145
Epoch: 15, Cost = 0.042254
Learning finished


In [11]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    model.eval()  # set evaluation mode (dropout=False)
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.targets[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9803999662399292
Label:  6
Prediction:  6
